#### Валерия Бунтякова

## 1: описание методов

1. Составить словарь сущностей. Данные: названия товаров. Плюсы: достаточно быстро и очень просто. Минусы: вынимаются лишние штуки.
2. Теггинг последовательностей. Данные: тексты отзывов или типа того. Плюсы: кажется это очень хорошо работает + без учителя. Минусы: надо много данных.
3. :о)

## 2: поиск именованых сущностей

In [1]:
from collections import Counter
import random
import re
import pandas as pd
import gzip

In [2]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')


In [3]:
# я использовала датасет с офисными принадлежностями
reviews = getDF('reviews_Office_Products_5.json.gz')
reviews = reviews[['reviewerID', 'reviewText']]

In [ ]:
# meta = getDF('meta_Office_Products.json.gz')

Сначала я хотела вынимать именованные сущности с помощью yargy, как мы делали на семинаре. Потом я поняла, что названия амазоне составлены как бог на душу положит, и ничего умного с ними сделать особо нельзя. Поэтому я стала выписывать в список категории товаров, пока список не начал покрывать >90% названий товаров. 

In [4]:
list_of_entities = ['cartridge', 'envelope', 'strip',
                   'stool', 'card', 'wallet',
                   'purse', 'shipping label', 'staples',
                    'key holder', 'planner', 'fountain pen ink',
                   'toner', 'plaque', 'monitor stand',
                   'calendar', 'badge reel',
                   'border', 'padfolio', 'key cover',
                   'paper', 'envelope', 'binder',
                   'pen', 'stapler', 'printer',
                    'film', 'template', 'poster',
                   'workstation', 'keyboard', 'door stop',
                   'notebook', 'postcard', 'book', 
                   'scanner', 'stapler', 'ink',
                   'sheet', 'cover', 'desk',
                   'feet hammock', 'tape', 'pencil',
                   'composition book', 'chair', 'laptop stand', 
                   'tablet stand', 'cd', 'lamp', 
                   'doorstop', 'stamp', 'eraser',
                   'adhesive', 'calculator', 'note',
                   'badge', 'lanyard', 'bag',
                   'clip', 'bookmark', 'projector', 
                   'letter opener', 'opener', 'letter',
                   'dictionary', 'photo paper',
                   'sign', 'table', 'badge rack', 
                   'rack', 'cork board', 'board', 
                   'crane', 'holder', 'phone', 
                    'toner', 'sharpener', 'keychain',
                   'sticker', 'marker', 'separator',
                   'notetab', 'laminator', 'pouch',
                   'tab', 'name tag', 'tag',
                   'moving set', 'label', 'mouse', 'pad',
                   'note', 'mouse', 'bulb',
                   'refill', 'pin', 'post-it',
                   'flag', 'folder', 'air bubble roll',
                   'diary', 'map', 'flag', 
                    'magnet', 'organizer', 'file',
                   'cd-rom']

## 3: вынуть биграммы

Штука которая вынимает биграммы. Можно ли было сделать это не регуляркой, и было бы лучше? Да. Когда начал пользоваться регулярками сложно остановится. 

In [5]:
r = re.compile('(?P<before>[\w-]+)(?: |\.|,)+(' + '|'.join(list_of_entities) + ')(?:ing|s)*(?: |\.|,)+(?P<after>[\w-]+)')

Уберем стопслова, чтобы биграммы получились более осмысленными, а не только "the <название товара>"

In [6]:
from nltk.tokenize import word_tokenize

In [7]:
with open('english_stopwords.txt', 'r') as file:
    stopwords = file.read().split()
    stopwords = stopwords + ['the',]

In [8]:
revs = reviews.reviewText.values

Будем считать размер корпуса в словах и хранить частотный словарь слов, чтобы потом можно было использовать это при подсчете метрик.

In [9]:
corpus_len = 0
word_dict = Counter()
clean = []
for rev in revs:
    new = [w.lower() for w in word_tokenize(rev) if w.lower() not in stopwords]
    corpus_len += len(new)
    for word in new:
        word_dict[word] += 1
    clean.append(' '.join(new))

In [10]:
del revs, reviews

In [11]:
corpus_len

5015474

In [12]:
collocations = []
n = 0
for c in clean:
    found = re.findall(r, c)
    if found:
        n += 1
    for f in found:
        collocations.append((f[0], f[1]))
        collocations.append((f[1], f[2]))
print(n)

47477


In [13]:
col_count = Counter(collocations)

In [14]:
col_count[('ink', 'cartridge')]

1018

## 4: коллокационные метрики

In [15]:
from math import sqrt, log

In [16]:
def t_score(bigram, counter, word_dict, corpus_len):
    bigr_freq = counter[bigram] / corpus_len
    
    w1 = bigram[0]
    w2 = bigram[1]
    
    p1 = word_dict[w1] / corpus_len
    p2 = word_dict[w2] / corpus_len
    
    return (bigr_freq - p1 * p2) / sqrt(bigr_freq/corpus_len)

In [17]:
def pmi(bigram, counter, word_dict, corpus_len):
    w1 = bigram[0]
    w2 = bigram[1]
    
    p1 = word_dict[w1] / corpus_len
    p2 = word_dict[w2] / corpus_len
    
    return log(counter[bigram]/(p1*p2), 2)

In [18]:
def dice(bigram, counter, word_dict):
    w1 = bigram[0]
    w2 = bigram[1]
    
    return 2*(counter[bigram]) / (word_dict[w1] + word_dict[w2])

Запишем в словарь значения метрик для кажой коллокации

In [19]:
def do_nothing():
    return

In [20]:
d = {}
for bigram, freq in col_count.items():
    if freq > 3:
        try:
            ts_ = t_score(bigram, col_count, word_dict, corpus_len)
            pmi_ = pmi(bigram, col_count, word_dict, corpus_len)
            dice_ = dice(bigram, col_count, word_dict)
            d[bigram] = {'freq':freq, 't-score':ts_, 'pmi':pmi_, 'dice':dice_}
        except:
            # если мы вынули коллокацию с какой-то штукой, которой нет в словаре
            do_nothing()

Чтобы найти лучшую метрику, посчитаем, что получается при разных метриках для одного и того же слова (pencil). Выберем сочетания, которые были бы нам полезны (good pencil, nice pencil) и посмотрим, как они ранжируются. 

In [21]:
t_score_pencil = [pair for pair in 
                  sorted(d.keys(), key=lambda x: d[x]['t-score'], reverse=True) 
                  if pair[1] == 'pencil' or pair[0] == 'pencil'] 
pmi_pencil = [pair for pair in 
                  sorted(d.keys(), key=lambda x: d[x]['pmi'], reverse=True) 
                  if pair[1] == 'pencil' or pair[0] == 'pencil'] 
dice_pencil = [pair for pair in 
                  sorted(d.keys(), key=lambda x: d[x]['dice'], reverse=True) 
                  if pair[1] == 'pencil' or pair[0] == 'pencil'] 

In [22]:
dice_score, pmi_score, ts_score = 0, 0, 0

for word in ['good', 'nice', 'great', 'expensive', 'cheap']:
    col = (word, 'pencil')
    ts_score += t_score_pencil.index(col)
    pmi_score += pmi_pencil.index(col)
    dice_score += dice_pencil.index(col)
    
print('t-score: ', ts_score)
print('pmi: ', pmi_score)
print('dice: ', dice_score)

t-score:  1075
pmi:  2214
dice:  887


Выбираем ту метрику, которая показывает меньшую сумму индексов, то есть выводит подходящие коллокации выше. Берем Дайса!

## 5: вывести примеры

In [23]:
# pencil
[pair for pair in 
 sorted(d.keys(), key=lambda x: d[x]['dice'], reverse=True) 
 if pair[1] == 'pencil' or pair[0] == 'pencil'][:10]

[('pencil', 'sharpener'),
 ('mechanical', 'pencil'),
 ('colored', 'pencil'),
 ('electric', 'pencil'),
 ('pencil', 'sharpeners'),
 ('sharpen', 'pencil'),
 ('sharpening', 'pencil'),
 ('pencil', 'n'),
 ('pencil', 'sharpened'),
 ('sharpens', 'pencil')]

In [24]:
# printer
[pair for pair in 
 sorted(d.keys(), key=lambda x: d[x]['dice'], reverse=True) 
 if pair[1] == 'printer' or pair[0] == 'printer'][:10]

[('laser', 'printer'),
 ('inkjet', 'printer'),
 ('photo', 'printer'),
 ('epson', 'printer'),
 ('hp', 'printer'),
 ('printer', 'n'),
 ('canon', 'printer'),
 ('all-in-one', 'printer'),
 ('wireless', 'printer'),
 ('new', 'printer')]

In [25]:
# paper
[pair for pair in 
 sorted(d.keys(), key=lambda x: d[x]['dice'], reverse=True) 
 if pair[1] == 'paper' or pair[0] == 'paper'][:10]

[('sheet', 'paper'),
 ('paper', 'tray'),
 ('piece', 'paper'),
 ('paper', 'n'),
 ('plain', 'paper'),
 ('pieces', 'paper'),
 ('paper', 'trays'),
 ('quality', 'paper'),
 ('regular', 'paper'),
 ('stack', 'paper')]

In [26]:
# envelope
[pair for pair in 
 sorted(d.keys(), key=lambda x: d[x]['dice'], reverse=True) 
 if pair[1] == 'envelope' or pair[0] == 'envelope'][:10]

[('envelope', 'n'),
 ('security', 'envelope'),
 ('padded', 'envelope'),
 ('seal', 'envelope'),
 ('reveal-n-seal', 'envelope'),
 ('sealing', 'envelope'),
 ('licking', 'envelope'),
 ('mailing', 'envelope'),
 ('manila', 'envelope'),
 ('10', 'envelope')]

In [27]:
# note
[pair for pair in 
 sorted(d.keys(), key=lambda x: d[x]['dice'], reverse=True) 
 if pair[1] == 'note' or pair[0] == 'note'][:10]

[('sticky', 'note'),
 ('post-it', 'note'),
 ('post', 'note'),
 ('taking', 'note'),
 ('note', 'n'),
 ('take', 'note'),
 ('quick', 'note'),
 ('note', 'tabs'),
 ('note', 'taking'),
 ('write', 'note')]